# 7.Phần Nâng cao - Cải tiến mô hình

Phần này học viên sẽ tự thực hiện việc cải tiến kết quả dự đoán của mô hình để cho ra kết quả tốt hơn ở phần trước. Tức là điểm số trên Kaggle của học viên phải vượt qua **0.8246 (Public Score), > top 50%**. Một vài phương pháp được đề xuất để học viên lựa chọn là:

- Thay đổi phương pháp lựa chọn đặc trưng để chọn ra bộ đặc trưng tốt hơn

- Sử dụng phương pháp khác để xử lý vấn đề mất cân bằng dữ liệu

- Sử dụng các mô hình học máy khác hoặc các kỹ thuật kết hợp mô hình nâng cao 

- Tập trung vào việc cải tiến các mô hình Boosting

- Tinh chỉnh các tham số của mô hình

- Sử dụng các kỹ thuật K-Fold hoặc chia nhỏ tập train để tránh hiện tượng Overfitting

### Load các package cần thiết

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline

### Load tập dữ liệu

In [3]:
train = pd.read_csv('dataset/train.csv')
test = pd.read_csv('dataset/test.csv')

test_id = test['ID']
train.drop('ID', axis=1, inplace = True)
test.drop('ID', axis=1, inplace = True)

X_train = train.drop('TARGET', axis=1)
y_train = train['TARGET']

### Khám phá dữ liệu

In [4]:
X_train.shape, y_train.shape, test.shape

((76020, 369), (76020,), (75818, 369))

In [7]:
numerical_features = X_train.select_dtypes(include=[np.number]).columns
numerical_features


Index(['var3', 'var15', 'imp_ent_var16_ult1', 'imp_op_var39_comer_ult1',
       'imp_op_var39_comer_ult3', 'imp_op_var40_comer_ult1',
       'imp_op_var40_comer_ult3', 'imp_op_var40_efect_ult1',
       'imp_op_var40_efect_ult3', 'imp_op_var40_ult1',
       ...
       'saldo_medio_var29_ult3', 'saldo_medio_var33_hace2',
       'saldo_medio_var33_hace3', 'saldo_medio_var33_ult1',
       'saldo_medio_var33_ult3', 'saldo_medio_var44_hace2',
       'saldo_medio_var44_hace3', 'saldo_medio_var44_ult1',
       'saldo_medio_var44_ult3', 'var38'],
      dtype='object', length=369)

In [8]:
categories_features = X_train.select_dtypes(include=[np.object]).columns
categories_features

Index([], dtype='object')

In [10]:
X_train.isnull().sum().sum()

0

In [11]:
test.isnull().sum().sum()

0

In [12]:
y_train.value_counts()

0    73012
1     3008
Name: TARGET, dtype: int64

In [19]:
ratio = y_train.value_counts()[1] / y_train.value_counts().sum()
ratio

0.0395685345961589

Sau khi khám phá nhận thấy:
- Dữ liệu toàn bộ là các dữ liệu số
- Không có dữ liệu bị khuyết
- Là imbalanced data do có 73012 target là 0 trong khi chỉ có 3008 target là 1 chiếm tỉ lệ 3,9%

### Preprocessing Data

#### Loại bỏ các biến trùng, hoặc gần giống nhau, hoặc có tương quan cao

In [21]:
from sklearn.pipeline import Pipeline

from feature_engine.selection import (
    DropConstantFeatures,
    DropDuplicateFeatures,
    SmartCorrelatedSelection,
)

pipe = Pipeline([
    ('constant', DropConstantFeatures(tol=0.998)),
    ('duplicated', DropDuplicateFeatures()),
    ('correlation', SmartCorrelatedSelection(selection_method='variance')),
])

pipe.fit(X_train)
X_train = pipe.transform(X_train)
X_test = pipe.transform(test)

In [22]:
X_train.shape, X_test.shape

((76020, 81), (75818, 81))

#### Lựa chọn đặc trưng bằng Extra Trees Classifier

In [24]:
selected_features = {}

In [25]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

clf = ExtraTreesClassifier(random_state=1729)
clf.fit(X_train, y_train)
fs = SelectFromModel(estimator=clf , prefit=True)

selected_features['extra_trees_classifier'] = X_train.columns[fs.get_support()].tolist()

#### Lựa chọn đặc trưng bằng: RecursiveFeatureElimination

In [27]:
from sklearn.ensemble import GradientBoostingClassifier
from feature_engine.selection import RecursiveFeatureElimination

model = GradientBoostingClassifier(
    n_estimators=10,
    max_depth=2,
    random_state=10,
)
sel = RecursiveFeatureElimination(
    variables=None, # automatically evaluate all numerical variables
    estimator = model, # the ML model
    scoring = 'roc_auc', # the metric we want to evalute
    threshold = 0.0005, # the maximum performance drop allowed to remove a feature
    cv=2, # cross-validation
)
sel.fit(X_train, y_train)

selected_features['recursive_feature_elimination'] = sel.transform(X_train).columns.tolist()

#### Lựa chọn đặc trưng bằng: RecursiveFeatureAddition

In [28]:
from feature_engine.selection import RecursiveFeatureAddition
model = GradientBoostingClassifier(
    n_estimators=10,
    max_depth=2,
    random_state=10,
)
# Setup the RFA selector
rfa = RecursiveFeatureAddition(
    variables=None,  # automatically evaluate all numerical variables
    estimator=model,  # the ML model
    scoring='roc_auc',  # the metric we want to evalute
    threshold=0.0001,  # the minimum performance increase needed to select a feature
    cv=2,  # cross-validation
)
rfa.fit(X_train, y_train)

selected_features['recursive_feature_addition'] = rfa.transform(X_train).columns.tolist()